Yishun has received a negative reputation as “Crazy Town”, and property prices might have been impacted. Are Yishun flats the cheapest in the country?

1.To create a dummy variable for each town \
2.encode flat type to integers \
3.To calculate flat age
4.conduct regression

In [5]:
###Import packages
import pandas as pd
import requests
import numpy as np
import statsmodels as sm
import statsmodels.api as smf  
from statsmodels.formula.api import ols #formula guide: https://www.statsmodels.org/dev/examples/notebooks/generated/formulas.html

In [6]:
#global values

#resale fp
RSF_fp="Relevant_datasets/RSF90onw_wAddress.csv"



In [7]:
#Import RSF
RSF=pd.read_csv(RSF_fp,index_col=0,low_memory=False)

In [10]:
RSF['flat_type'].unique()
#to assume a linear relationship btw flat type and resale price.
#assume executive>multi-gen>5 room in general in terms of resale price

#Encode flat type to numerical data
RSF['flat_type_n']=0
RSF.loc[RSF['flat_type']=='1 ROOM','flat_type_n']=1
RSF.loc[RSF['flat_type']=='2 ROOM','flat_type_n']=2
RSF.loc[RSF['flat_type']=='3 ROOM','flat_type_n']=3
RSF.loc[RSF['flat_type']=='4 ROOM','flat_type_n']=4
RSF.loc[RSF['flat_type']=='5 ROOM','flat_type_n']=5
RSF.loc[RSF['flat_type']=='MULTI-GENERATION','flat_type_n']=6
RSF.loc[RSF['flat_type']=='MULTI GENERATION','flat_type_n']=6
RSF.loc[RSF['flat_type']=='EXECUTIVE','flat_type_n']=7
#check number of unchanged rows after encoding
print("no. of unchanged rows:",RSF.loc[RSF['flat_type_n']==0]['flat_type_n'].count())

no. of unchanged rows: 0


In [12]:
# to automate the writing of the dummy variable portion of the OLS formula to minimize error

#For easy reference down the line (i.e. when writing regression formula), to replace " " with "_" for town names
RSF['town'] = RSF['town'].str.replace(' ',"_")
RSF['town'] = RSF['town'].str.replace('/',"_")

#create a list of unique towns for easy reference (to be created before turning column to dummy variable)
town_list=list(RSF['town'].unique())

#rename each value in list to the name of its corresponding column by adding "town_"
town_list = ['town_' + x for x in town_list]

In [13]:
#create a dummy variable of towns
RSF_dummies = pd.get_dummies(RSF,columns=['town'])

#convert boolean to int for dummy variables to ease regression
for i in town_list:
   RSF_dummies[i] = RSF_dummies[i].astype(int)

In [18]:
## to automate the writing of the dummy variable portion of the OLS formula to minimize error

#remove Yishun from list before adding to formula to prevent multicollinearity (1 town needs to be absent during regression)
town_list.remove('town_YISHUN')

#checking if yishun is still in the list
if 'town_YISHUN' in town_list:
    print("Yishun is in the list")
else:
    print("Yishun is not in the list")

str_town = ' + '.join(town_list)

print(str_town) 

Yishun is not in the list
town_ANG_MO_KIO + town_BEDOK + town_BISHAN + town_BUKIT_BATOK + town_BUKIT_MERAH + town_BUKIT_TIMAH + town_CENTRAL_AREA + town_CHOA_CHU_KANG + town_CLEMENTI + town_GEYLANG + town_HOUGANG + town_JURONG_EAST + town_JURONG_WEST + town_KALLANG_WHAMPOA + town_MARINE_PARADE + town_QUEENSTOWN + town_SENGKANG + town_SERANGOON + town_TAMPINES + town_TOA_PAYOH + town_WOODLANDS + town_LIM_CHU_KANG + town_SEMBAWANG + town_BUKIT_PANJANG + town_PASIR_RIS + town_PUNGGOL


In [27]:
#Create a variable for flat age
RSF_dummies['flat_age']=RSF_dummies['Year_x']-RSF_dummies['lease_commence_date']

In [29]:
form=str('Real_resale_price ~ flat_type_n + floor_area_sqm + flat_age + '+str_town)
OLS3a = ols(formula=form, data=RSF_dummies).fit()

In [31]:
print(OLS3a.summary())

                            OLS Regression Results                            
Dep. Variable:      Real_resale_price   R-squared:                       0.799
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                 1.267e+05
Date:                Sun, 16 Jun 2024   Prob (F-statistic):               0.00
Time:                        23:18:17   Log-Likelihood:            -1.2035e+07
No. Observations:              927061   AIC:                         2.407e+07
Df Residuals:                  927031   BIC:                         2.407e+07
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept            -1.451e+05 

According to the regression if the dummy variable, after controlling for typical influences of real resale prices such as floor sqm and flat type, there are towns ("town_XXX") with statistically significant correlation coefficients that are negative (i.e. coef<0). That would mean that there are towns were the average prices of resale flats are cheaper than Yishun. Given that the regression table shows that there are towns (e.g. woodlands=-6.433e+04***,sembawang=-3.981e+04***) with statistically significant correlation coefficients<0, it would mean that relative to Yishun, they are cheaper on average. Hence there is no evidence from this analysis that Yishun flats are the cheapest in the country.strong multicollinearity or other numerical problems.